# Quiz 1

In [7]:
"""
Your task in this exercise is to modify 'extract_carrier()` to get a list of
all airlines. Exclude all of the combination values like "All U.S. Carriers"
from the data that you return. You should return a list of codes for the
carriers.

All your changes should be in the 'extract_carrier()' function. The
'options.html' file in the tab above is a stripped down version of what is
actually on the website, but should provide an example of what you should get
from the full file.

Please note that the function 'make_request()' is provided for your reference
only. You will not be able to to actually use it from within the Udacity web UI.
"""

from bs4 import BeautifulSoup
html_page = "data/options.html"

In [21]:
page = "data/options.html"
with open(page, "r") as html:
        soup = BeautifulSoup(html, "lxml")

In [27]:
carrier_list = soup.find('select', id='CarrierList')
for carrier in carrier_list.find_all('option'):
    print('{} \t\t\t-\t {}'.format(carrier.get('value'), carrier.string))

All 			-	 All U.S. and Foreign Carriers
AllUS 			-	 All U.S. Carriers
AllForeign 			-	 All Foreign Carriers
FL 			-	 AirTran Airways
AS 			-	 Alaska Airlines 
AA 			-	 American Airlines 
MQ 			-	 American Eagle Airlines 
5Y 			-	 Atlas Air 
DL 			-	 Delta Air Lines 
EV 			-	 ExpressJet Airlines 
F9 			-	 Frontier Airlines 
HA 			-	 Hawaiian Airlines 
B6 			-	 JetBlue Airways
OO 			-	 SkyWest Airlines 
WN 			-	 Southwest Airlines 
NK 			-	 Spirit Air Lines
US 			-	 US Airways 
UA 			-	 United Air Lines 
VX 			-	 Virgin America


In [31]:
def extract_carriers(page):

    with open(page, "r") as html:
        soup = BeautifulSoup(html, "lxml")
    carrier_list = soup.find('select', id='CarrierList')
    data = [c.get('value') 
        for c in carrier_list.find_all('option') 
        if not c.get('value').startswith('All')]

    return data

In [32]:
def make_request(data):
    eventvalidation = data["eventvalidation"]
    viewstate = data["viewstate"]
    airport = data["airport"]
    carrier = data["carrier"]

    r = s.post("https://www.transtats.bts.gov/Data_Elements.aspx?Data=2",
               data = (("__EVENTTARGET", ""),
                       ("__EVENTARGUMENT", ""),
                       ("__VIEWSTATE", viewstate),
                       ("__VIEWSTATEGENERATOR",viewstategenerator),
                       ("__EVENTVALIDATION", eventvalidation),
                       ("CarrierList", carrier),
                       ("AirportList", airport),
                       ("Submit", "Submit")))

    return r.text


def test():
    data = extract_carriers(html_page)
    assert len(data) == 16
    assert "FL" in data
    assert "NK" in data

test()

# Quiz 2

In [36]:
"""
Complete the 'extract_airports()' function so that it returns a list of airport
codes, excluding any combinations like "All".

Refer to the 'options.html' file in the tab above for a stripped down version
of what is actually on the website. The test() assertions are based on the
given file.
"""

from bs4 import BeautifulSoup
html_page = "data/options.html"

In [43]:
def extract_airports(page):
    with open(page, "r") as html:
        soup = BeautifulSoup(html, "lxml")
        airport_list = soup.find('select', id='AirportList')
        data = [c.get('value') 
                for c in airport_list.find_all('option') 
                if not c.get('value').startswith('All')]

    return data

In [44]:
extract_airports(html_page)

['ATL',
 'BWI',
 'BOS',
 'CLT',
 'MDW',
 'ORD',
 'DFW',
 'DEN',
 'DTW',
 'FLL',
 'IAH',
 'LAS',
 'LAX',
 'ABR',
 'ABI']

In [45]:
def test():
    data = extract_airports(html_page)
    assert len(data) == 15
    assert "ATL" in data
    assert "ABR" in data

test()

# Quiz 3

In [110]:
"""
Let's assume that you combined the code from the previous 2 exercises with code
from the lesson on how to build requests, and downloaded all the data locally.
The files are in a directory "data", named after the carrier and airport:
"{}-{}.html".format(carrier, airport), for example "FL-ATL.html".

The table with flight info has a table class="dataTDRight". Your task is to
use 'process_file()' to extract the flight data from that table as a list of
dictionaries, each dictionary containing relevant data from the file and table
row. This is an example of the data structure you should return:

data = [{"courier": "FL",
         "airport": "ATL",
         "year": 2012,
         "month": 12,
         "flights": {"domestic": 100,
                     "international": 100}
        },
         {"courier": "..."}
]

Note - year, month, and the flight data should be integers.
You should skip the rows that contain the TOTAL data for a year.

There are couple of helper functions to deal with the data files.
Please do not change them for grading purposes.
All your changes should be in the 'process_file()' function.

The 'data/FL-ATL.html' file in the tab above is only a part of the full data,
covering data through 2003. The test() code will be run on the full table, but
the given file should provide an example of what you will get.
"""
from bs4 import BeautifulSoup
from zipfile import ZipFile
import os

datadir = "data/response"


def open_zip(datadir):
    with ZipFile('{0}.zip'.format(datadir), 'r') as myzip:
        myzip.extractall()

        
def process_all(datadir):
    files = os.listdir(datadir)
    return files

In [111]:
f = 'FL-ATL.html'
data = []
info = {}
info["courier"], info["airport"] = f[:6].split("-")
# Note: create a new dictionary for each entry in the output data list.
# If you use the info dictionary defined here each element in the list 
# will be a reference to the same info dictionary.
with open("{}/{}".format(datadir, f), "r") as html:
    soup = BeautifulSoup(html, 'lxml')


In [112]:
table = soup.find('table', id='DataGrid1')

In [113]:
def extract_row(row, info):
    values = [int(v.string.replace(',', '')) if v.string != 'TOTAL' else v.string
              for v in row.find_all('td')]
    if values[1] == 'TOTAL':
        return None
    info['year'] = values[0]
    info['month'] = values[1]
    info['flights'] = {'domestic': values[2],
                       'international': values[3]}
    return info

In [114]:
info

{'courier': 'FL', 'airport': 'ATL'}

In [115]:
for row in table.find_all('tr', {'class':'dataTDRight'}):
    print(extract_row(row, info))

{'courier': 'FL', 'airport': 'ATL', 'year': 2002, 'month': 10, 'flights': {'domestic': 815489, 'international': 92565}}
{'courier': 'FL', 'airport': 'ATL', 'year': 2002, 'month': 11, 'flights': {'domestic': 766775, 'international': 91342}}
{'courier': 'FL', 'airport': 'ATL', 'year': 2002, 'month': 12, 'flights': {'domestic': 782175, 'international': 96881}}
None
{'courier': 'FL', 'airport': 'ATL', 'year': 2003, 'month': 1, 'flights': {'domestic': 785651, 'international': 98053}}
{'courier': 'FL', 'airport': 'ATL', 'year': 2003, 'month': 2, 'flights': {'domestic': 690750, 'international': 85965}}
{'courier': 'FL', 'airport': 'ATL', 'year': 2003, 'month': 3, 'flights': {'domestic': 797634, 'international': 97929}}
{'courier': 'FL', 'airport': 'ATL', 'year': 2003, 'month': 4, 'flights': {'domestic': 766639, 'international': 89398}}
{'courier': 'FL', 'airport': 'ATL', 'year': 2003, 'month': 5, 'flights': {'domestic': 789857, 'international': 87671}}
{'courier': 'FL', 'airport': 'ATL', 'yea

In [120]:
def process_file(f):
    """
    This function extracts data from the file given as the function argument in
    a list of dictionaries. This is example of the data structure you should
    return:

    data = [{"courier": "FL",
             "airport": "ATL",
             "year": 2012,
             "month": 12,
             "flights": {"domestic": 100,
                         "international": 100}
            },
            {"courier": "..."}
    ]


    Note - year, month, and the flight data should be integers.
    You should skip the rows that contain the TOTAL data for a year.
    """
    def extract_row(row, info):
        values = [int(v.string.replace(',', '')) 
                  if v.string != 'TOTAL' else v.string
                  for v in row.find_all('td')]
        if values[1] == 'TOTAL':
            return None
        info['year'] = values[0]
        info['month'] = values[1]
        info['flights'] = {'domestic': values[2],
                           'international': values[3]}
        return info
    
    info = {}
    info["courier"], info["airport"] = f[:6].split("-")
    # Note: create a new dictionary for each entry in the output data list.
    # If you use the info dictionary defined here each element in the list 
    # will be a reference to the same info dictionary.
    with open("{}/{}".format(datadir, f), "r") as html:
        soup = BeautifulSoup(html, 'lxml')
    table = soup.find('table', id='DataGrid1')
    
    data = [extract_row(row, info)
            for row in table.find_all('tr', {'class':'dataTDRight'})
            if extract_row(row, info)]
    
    return data

In [121]:
process_file('FL-ATL.html')

[{'courier': 'FL',
  'airport': 'ATL',
  'year': 2003,
  'month': 12,
  'flights': {'domestic': 798879, 'international': 97094}},
 {'courier': 'FL',
  'airport': 'ATL',
  'year': 2003,
  'month': 12,
  'flights': {'domestic': 798879, 'international': 97094}},
 {'courier': 'FL',
  'airport': 'ATL',
  'year': 2003,
  'month': 12,
  'flights': {'domestic': 798879, 'international': 97094}},
 {'courier': 'FL',
  'airport': 'ATL',
  'year': 2003,
  'month': 12,
  'flights': {'domestic': 798879, 'international': 97094}},
 {'courier': 'FL',
  'airport': 'ATL',
  'year': 2003,
  'month': 12,
  'flights': {'domestic': 798879, 'international': 97094}},
 {'courier': 'FL',
  'airport': 'ATL',
  'year': 2003,
  'month': 12,
  'flights': {'domestic': 798879, 'international': 97094}},
 {'courier': 'FL',
  'airport': 'ATL',
  'year': 2003,
  'month': 12,
  'flights': {'domestic': 798879, 'international': 97094}},
 {'courier': 'FL',
  'airport': 'ATL',
  'year': 2003,
  'month': 12,
  'flights': {'domes

# Quiz 4

In [125]:
"""
This and the following exercise are using US Patent database. The patent.data
file is a small excerpt of much larger datafiles that are available for
download from US Patent website. These files are pretty large ( >100 MB each).
The original file is ~600MB large, you might not be able to open it in a text
editor.

The data itself is in XML, however there is a problem with how it's formatted.
Please run this script and observe the error. Then find the line that is
causing the error. You can do that by just looking at the datafile in the web
UI, or programmatically. For quiz purposes it does not matter, but as an
exercise we suggest that you try to do it programmatically.

NOTE: You do not need to correct the error - for now, just find where the error
is occurring.
"""

import xml.etree.ElementTree as ET

PATENTS = 'data/patent.data'

def get_root(fname):

    tree = ET.parse(fname)
    return tree.getroot()


get_root(PATENTS)

ParseError: junk after document element: line 657, column 0 (<string>)

In [129]:
with open(PATENTS) as file:
    for i in range(655):
        file.readline()
    for i in range(3):
        print('L{}: {}'.format(i, file.readline()))

L0: </us-patent-grant>

L1: <?xml version="1.0" encoding="UTF-8"?>

L2: <!DOCTYPE us-patent-grant SYSTEM "us-patent-grant-v44-2013-05-16.dtd" [ ]>



# Quiz 6

In [137]:
# So, the problem is that the gigantic file is actually not a valid XML, because
# it has several root elements, and XML declarations.
# It is, a matter of fact, a collection of a lot of concatenated XML documents.
# So, one solution would be to split the file into separate documents,
# so that you can process the resulting files as valid XML documents.

import xml.etree.ElementTree as ET
PATENTS = 'data/patent.data'

def get_root(fname):
    tree = ET.parse(fname)
    return tree.getroot()

In [150]:
def split_file(filename):
    """
    Split the input file into separate files, each containing a single patent.
    As a hint - each patent declaration starts with the same line that was
    causing the error found in the previous exercises.
    
    The new files should be saved with filename in the following format:
    "{}-{}".format(filename, n) where n is a counter, starting from 0.
    """
    fname = filename.split('/')[-1]
    with open(filename, 'r') as file:
        line = file.readline()
        file_content = ''
        header = line
        file_idx = 0
        while line:
            file_content += line
            line = file.readline()
            if line == header:
                with open('{}-{}'.format(fname, file_idx), 'w') as output:
                    output.write(file_content)
                file_idx +=1
                file_content = ''

        # Write the last one
        with open('{}-{}'.format(fname, file_idx), 'w') as output:
            output.write(file_content)

In [151]:
def test():
    split_file(PATENTS)
    for n in range(4):
        try:
            fname = "{}-{}".format(PATENTS, n)
            f = open(fname, "r")
            if not f.readline().startswith("<?xml"):
                print("You have not split the file {} in the correct boundary!".format(fname))
            f.close()
        except:
            print("Could not find file {}. Check if the filename is correct!".format(fname))

test()